In [ ]:
!pip install pdf2image
!pip install python-doctr
!apt-get install poppler-utils
!pip install jiwer
!pip install tf2onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 13.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1

In [ ]:
import os
import re
from pdf2image import convert_from_path
import numpy as np
from jiwer import wer, cer
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

def clean_text(text):
    """
    Remove invalid XML characters from the text.
    """
    invalid_xml_chars = re.compile(u'[\u0000-\u0008\u000B-\u000C\u000E-\u001F]')
    return invalid_xml_chars.sub('', text)


directory_path = 'text/'
model = ocr_predictor(pretrained=True)

for filename in os.listdir(directory_path):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(directory_path, filename)
        ground_truth_path = os.path.join(directory_path, filename.replace('.pdf', '.txt'))

        if not os.path.exists(ground_truth_path):
            print(f"Skipping {filename} as ground truth text file does not exist.")
            continue
        doc = DocumentFile.from_pdf(pdf_path)

        # Initialize variable to hold all OCR output text
        all_text = []

        # Perform OCR on each page
        for page in doc:
            # Perform OCR using DocTR
            result = model([page])
            page_text = ' '.join([word.value for block in result.pages[0].blocks for line in block.lines for word in line.words])
            clean_text_data = clean_text(page_text)
            all_text.append(clean_text_data)

        # Concatenate all OCR output text into a single string
        ocr_output_text = '\n'.join(all_text)

        # Load ground truth text
        with open(ground_truth_path, 'r', encoding='utf-8') as gt_file:
            ground_truth_text = gt_file.read()

        # Calculate WER and CER
        calculated_wer = wer(ground_truth_text, ocr_output_text)
        calculated_cer = cer(ground_truth_text, ocr_output_text)

        print(f"Results for {filename}:")
        print(f"\nWER: {calculated_wer:.2f}")
        print(f"CER: {calculated_cer:.2f}\n")


  0%|          | 0/94122142 [00:00<?, ?it/s]

DEBUG:tensorflow:Layer lstm will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm_1 will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm_1 will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm_1 will use cuDNN kernels when running on GPU.


  0%|          | 0/58758994 [00:00<?, ?it/s]

Results for 2EDEPZ4VHTLPTWSZR6FAVUJ3B2ZVSIPS.pdf:

WER: 0.20
CER: 0.14



In [ ]:
#Metadata extraction based on predefined keywords
import os
import re
import xml.etree.ElementTree as ET
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

def clean_text(text):
    """
    Remove invalid XML characters from the text.
    """
    invalid_xml_chars = re.compile(u'[\u0000-\u0008\u000B-\u000C\u000E-\u001F]')
    return invalid_xml_chars.sub('', text)

def save_to_xml(doc_text, file_path):
    """
    Saves the OCR text in a structured XML format with 'Document', 'Page', and 'Line' elements.
    """
    root = ET.Element("Document")
    for page_number, page_content in enumerate(doc_text, start=1):
        page_elem = ET.SubElement(root, "Page", number=str(page_number))
        for line_number, line in enumerate(page_content, start=1):
            line_elem = ET.SubElement(page_elem, "Line", number=str(line_number))
            line_elem.text = line

    tree = ET.ElementTree(root)
    tree.write(file_path, encoding="utf-8", xml_declaration=True)

def search_in_xml(file_path, keywords):
    tree = ET.parse(file_path)
    root = tree.getroot()
    found_keywords = {}
    keyword_iter = iter(keywords)
    current_keyword = next(keyword_iter, None)
    capture_text = False
    collected_text = []

    for page in root.findall('Page'):
        for line in page.findall('Line'):
            text = line.text.strip()
            if current_keyword and current_keyword in text:
                if capture_text and collected_text:
                    # Save the collected text before starting a new keyword
                    found_keywords[previous_keyword] = " ".join(collected_text)
                    collected_text = []
                # Start capturing text from the next line
                capture_text = True
                previous_keyword = current_keyword
                current_keyword = next(keyword_iter, None)
                continue

            if capture_text:
                collected_text.append(text)

    # Handle the last keyword separately
    if collected_text:
        if previous_keyword == keywords[-1]:
            complete_text = " ".join(collected_text)
            found_keywords[previous_keyword] = complete_text[:41] if len(complete_text) > 41 else complete_text
        else:
            found_keywords[previous_keyword] = " ".join(collected_text)

    return found_keywords

directory_path = 'data/'
keywords = ["Fastighetsagare", "Namn", "Adress", "Postnr och ort", "Fastighetsbeteckning", "Anlaggare av ledning"]
# Create a DocTR OCR predictor - specify the model as needed
model = ocr_predictor(pretrained=True)

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(directory_path, filename)
        xml_path = os.path.join(directory_path, filename.replace('.pdf', '.xml'))

        # Load PDF document
        doc = DocumentFile.from_pdf(pdf_path)

        # Initialize variable to hold all OCR output text for each page
        all_pages_text = []

        # Perform OCR on each page
        for page in doc:
            page_text = []
            # Perform OCR using DocTR
            result = model([page])
            for block in result.pages[0].blocks:
                for line in block.lines:
                    line_text = ' '.join(word.value for word in line.words)
                    clean_line = clean_text(line_text)
                    page_text.append(clean_line)
            all_pages_text.append(page_text)
        save_to_xml(all_pages_text, xml_path)

        # Search for keywords in XML
        keyword_results = search_in_xml(xml_path, keywords)

        # Print each keyword and its value in a simplified format
        for key, value in keyword_results.items():
            print(f"{key}: {value}")

        print(f"Processed and saved: {xml_path}")


DEBUG:tensorflow:Layer lstm_8 will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm_8 will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm_8 will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm_9 will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm_9 will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm_9 will use cuDNN kernels when running on GPU.


Fastighetsagare: Tohns.LAmBE2 4
Namn: ARNBERE.. 11
Adress: 95.9. NOZS 30
Postnr och ort: Norsto Norsjo kommun Storgatan 67 935 81 NORSJO Organisationsnr: 212000-2858 -
Anlaggare av ledning: Med underjordisk ledning for data- och te
Processed and saved: data/3.xml


In [ ]:
!unzip table-filename.zip


In [ ]:
!zip -r table-filename.zip table

metria/  sample_data/  table-filename.zip


In [ ]:
#Calculate time duration
import os
import re
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from pdf2image import convert_from_path
from doctr.models import ocr_predictor
from PIL import Image as PILImage
import time

def clean_text(text):
    """
    Remove invalid XML characters from the text.
    """
    invalid_xml_chars = re.compile(u'[\u0000-\u0008\u000B-\u000C\u000E-\u001F]')
    return invalid_xml_chars.sub('', text)

def preprocess_image(page):
    """
    Convert page to grayscale and apply thresholding to enhance OCR accuracy.
    """
    gray = cv2.cvtColor(page, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    three_channel = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
    return three_channel

def convert_pdf_to_images(pdf_path):
    """
    Converts a PDF file into a list of images, one per page.
    """
    return convert_from_path(pdf_path, dpi=200)

def save_to_xml(doc_text, file_path):
    """
    Saves the OCR text in a structured XML format with 'Document', 'Page', and 'Line' elements.
    """
    root = ET.Element("Document")
    for page_number, page_content in enumerate(doc_text, start=1):
        page_elem = ET.SubElement(root, "Page", number=str(page_number))
        for line_number, line in enumerate(page_content, start=1):
            line_elem = ET.SubElement(page_elem, "Line", number=str(line_number))
            line_elem.text = line
    tree = ET.ElementTree(root)
    tree.write(file_path, encoding="utf-8", xml_declaration=True)

def search_in_xml(file_path, keywords):
    """
    Searches for occurrences of keywords in an XML file and captures text between keywords.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()
    found_keywords = {}
    current_keyword = None
    collected_text = []
    for page in root.findall('Page'):
        for line in page.findall('Line'):
            text = line.text.strip()
            if current_keyword:
                collected_text.append(text)
            for keyword in keywords:
                if keyword in text and keyword not in found_keywords:
                    if current_keyword:
                        found_keywords[current_keyword] = " ".join(collected_text)
                        collected_text = []
                    current_keyword = keyword
                    break
    if current_keyword:
        found_keywords[current_keyword] = " ".join(collected_text)
    return found_keywords

def process_files(directory_path):
    """
    Process each PDF in the directory.
    """
    start_total = time.time()
    keywords = ["Fastighetsagare", "Namn", "Adress", "Postnr och ort", "Fastighetsbeteckning", "Anlaggare av ledning"]
    model = ocr_predictor(pretrained=True)

    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            start_time = time.time()

            pdf_path = os.path.join(directory_path, filename)
            xml_path = os.path.join(directory_path, filename.replace('.pdf', '.xml'))
            images = convert_pdf_to_images(pdf_path)
            all_pages_text = []

            for image in images:
                np_image = np.array(image)
                np_image = np_image[:, :, ::-1]  # Convert from PIL's RGB to OpenCV's BGR
                preprocessed_image = preprocess_image(np_image)
                result = model([preprocessed_image])

                page_text = []
                for block in result.pages[0].blocks:
                    for line in block.lines:
                        line_text = ' '.join(word.value for word in line.words)
                        clean_line = clean_text(line_text)
                        page_text.append(clean_line)
                all_pages_text.append(page_text)

            save_to_xml(all_pages_text, xml_path)
            keyword_results = search_in_xml(xml_path, keywords)

            for key, value in keyword_results.items():
                print(f"{key}: {value}")

            end_time = time.time()
            print(f"Processed and saved: {xml_path}, Time taken: {end_time - start_time} seconds")

    end_total = time.time()
    print(f"Total processing time for all files: {end_total - start_total} seconds")

if __name__ == "__main__":
    directory_path = 'data/'
    process_files(directory_path)


  0%|          | 0/94122142 [00:00<?, ?it/s]

DEBUG:tensorflow:Layer lstm will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm_1 will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm_1 will use cuDNN kernels when running on GPU.
DEBUG:tensorflow:Layer lstm_1 will use cuDNN kernels when running on GPU.


  0%|          | 0/58758994 [00:00<?, ?it/s]

Fastighetsagare: TpMnsLumBE2 G Namn
Namn: ARNBERG. I1. Adress
Adress: %5.9. NOZS 30 Postnr och ort
Postnr och ort: No2S16 Norsjo kommun Storgatan 67 935 81 NORSJO Organisationsnr: 212000-2858 4:2 Fastighetsbeteckning:
Fastighetsbeteckning: Anlaggare av ledning
Anlaggare av ledning: Med underjordisk ledning for data- och telekommunikation enligt detta avtal avses alla kablar och ledningar genom vilka Overfors signaler for t ex bild, data eller annat. I underjordisk ledning ingàr sâdana for dess andamal erforderliga anordningar som brunnar, Fastighetsàgaren ger anlaggaren râtt till markutrymme pa angiven fastighet for anlaggning, bibehâllande, underhall och nyttjande av underjordiska ledningar for data- och telekommunikation samt disponera det utrymme som behovs for att utfora arbete enligt den Vid byte av anlaggningsagare overgàr nyttjanderatten automatiskt till den nye Anlaggningsarbetet ska pâborjas under tiden 2017-05-15 2017-10-30. skarvlador, tomror, skâp och andra tillbehor. strac